#### Samples can be find there

https://github.com/Azure-Samples/azure-search-python-samples/blob/main/Quickstart-Agentic-Retrieval/quickstart-agentic-retrieval.ipynb

In [43]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.search.documents.knowledgebases import KnowledgeBaseRetrievalClient
from azure.search.documents.knowledgebases.models import KnowledgeBaseRetrievalRequest, KnowledgeBaseMessage, KnowledgeBaseMessageTextContent, SearchIndexKnowledgeSourceParams
from azure.search.documents.indexes.models import KnowledgeRetrievalLowReasoningEffort, KnowledgeRetrievalMediumReasoningEffort
from IPython.display import display, Markdown
from typing import List
import os
import json

In [18]:
load_dotenv(override=True)

search_endpoint = os.getenv('SEARCH_ENDPOINT')
search_key = os.getenv('SEARCH_API_KEY')
knowledge_base_name = os.getenv('KNOWLEDGE_BASE_NAME')
knowledge_sources_name = os.getenv('KNOWLEDGE_SOURCES_NAME')

In [16]:
credential = DefaultAzureCredential()
agent_client = KnowledgeBaseRetrievalClient(endpoint=search_endpoint,
                                            knowledge_base_name=knowledge_base_name,
                                            credential=credential)

In [6]:
instructions = """
You are a Q&A expert specializing in Earth at night phenomena and data.

## Your Role
Provide accurate, factual answers about Earth at night based on the available knowledge base. Your responses should be informative, clear, and grounded in the documented information.

## Response Guidelines

1. **Answer from Knowledge**: Base all answers on information retrieved from the knowledge base. Only provide information you can verify from the available sources.

2. **Handle Uncertainty**: If information isn't available in the knowledge base, respond with: "I don't have that information in my current knowledge base."

3. **Be Specific**: When answering, provide:
   - Direct, clear explanations
   - Relevant details and context
   - Specific data points when available

4. **Stay Focused**: Only answer questions related to Earth at night topics such as:
   - Nighttime satellite imagery and observations
   - Light pollution and urban illumination patterns
   - Nocturnal environmental phenomena
   - Dark sky preservation
   - Night-time Earth science data

## What You Can Help With
- Questions about night-time Earth observations
- Data and patterns visible from space at night
- Scientific information about Earth's night-side phenomena
- Historical or comparative night-time data

## What You Cannot Help With
❌ Topics unrelated to Earth at night
❌ Real-time or current data requests (only documented knowledge)
❌ Speculation beyond available information

If you don't have the answer, respond with "I don't know".
"""

messages = [
    {
        "role": "system",
        "content": instructions
    }
]

In [33]:
query = "What's the bounty for Bjorn Iron-Fist and what penalties does he face for his crime?"

messages.append({
    "role": "user",
    "content": query
})

In [34]:
ks = knowledge_sources_name.split(',')
params: List[SearchIndexKnowledgeSourceParams] = []
for k in ks:
   p = SearchIndexKnowledgeSourceParams(
      knowledge_source_name=k,
      include_references=True,
      include_reference_source_data=True,
      always_query_source=True
   )
   params.append(p)

In [35]:
req = KnowledgeBaseRetrievalRequest(
    messages=[
        KnowledgeBaseMessage(
            role=m["role"],
            content=[KnowledgeBaseMessageTextContent(text=m["content"])]
        ) for m in messages
    ],
    knowledge_source_params=params,
    include_activity=True,
    retrieval_reasoning_effort=KnowledgeRetrievalMediumReasoningEffort
)

In [29]:
response_contents = []
activity_contents = []
references_contents = []

In [36]:
result = agent_client.retrieve(retrieval_request=req)
print(f"Retrieved content from '{knowledge_base_name}' successfully.")

Retrieved content from 'kb-crime-offence' successfully.


In [ ]:
# Build simple string values for response_content, activity_content, and references_content

# Responses -> Concatenate text/value fields from all response contents
response_parts = []
for resp in result.response:
    for content in resp.content:
        response_parts.append(content.text)
response_content = "\n\n".join(response_parts) if response_parts else "No response found on 'result'"

response_contents.append(response_content)

# Display with Markdown rendering
display(Markdown(f"### Response Content\n\n{response_content}"))

In [38]:
# Activity -> JSON string of activity as list of dicts
if result.activity:
    activity_content = json.dumps([a.as_dict() for a in result.activity], indent=2)
else:
    activity_content = "No activity found on 'result'"
    
activity_contents.append(activity_content)
print("activity_content:\n", activity_content, "\n")

activity_content:
 [
  {
    "id": 0,
    "type": "modelQueryPlanning",
    "elapsed_ms": 2558,
    "input_tokens": 1464,
    "output_tokens": 75
  },
  {
    "id": 1,
    "type": "azureBlob",
    "elapsed_ms": 739,
    "knowledge_source_name": "ks-bounty",
    "query_time": "2026-01-07T21:20:06.114Z",
    "count": 5,
    "azure_blob_arguments": {
      "search": "bounty for Bjorn Iron-Fist"
    }
  },
  {
    "id": 2,
    "type": "azureBlob",
    "elapsed_ms": 240,
    "knowledge_source_name": "ks-bounty",
    "query_time": "2026-01-07T21:20:06.363Z",
    "count": 2,
    "azure_blob_arguments": {
      "search": "penalties faced by Bjorn Iron-Fist for his crime"
    }
  },
  {
    "id": 3,
    "type": "azureBlob",
    "elapsed_ms": 348,
    "knowledge_source_name": "ks-crime-penality",
    "query_time": "2026-01-07T21:20:06.712Z",
    "count": 6,
    "azure_blob_arguments": {
      "search": "bounty for Bjorn Iron-Fist"
    }
  },
  {
    "id": 4,
    "type": "azureBlob",
    "elapsed

In [39]:
# References -> JSON string of references as list of dicts
if result.references:
    references_content = json.dumps([r.as_dict() for r in result.references], indent=2)
else:
    references_content = "No references found on 'result'"
    
references_contents.append(references_content)
print("references_content:\n", references_content)

references_content:
 [
  {
    "type": "azureBlob",
    "id": "0",
    "activity_source": 1,
    "reranker_score": 3.4292195,
    "blob_url": "https://strkggvd7uq3zxxx6.blob.core.windows.net/bounty/dossier_Bjorn_Iron-Fist.pdf"
  },
  {
    "type": "azureBlob",
    "id": "1",
    "activity_source": 3,
    "reranker_score": 2.6288283,
    "blob_url": "https://strkggvd7uq3zxxx6.blob.core.windows.net/crimepenalty/pettyTheft_penalties.pdf"
  }
]
